# Source Localization Workflow

In [2]:


# generated_closed = np.load('./data/training/eyes-closed-64ch.npy')
# generated_open = np.load('./data/training/eyes-open-64ch.npy')

# ch_names = ['EEG {0}'.format(i) for i in range(64)] 
# ch_types = ['eeg' for _ in range(64)]
# info = mne.create_info(ch_names=ch_names, sfreq=160, ch_types=ch_types)

# generated_closed_mne = []
# for i in range(len(generated_closed)):
#     generated_closed_mne.append(mne.io.RawArray(generated_closed[i], info))

# generated_open_mne = []
# for i in range(len(generated_open)):
#     generated_open_mne.append(mne.io.RawArray(generated_open[i], info))


In [9]:
# %pip install nibabel
# %pip install pyvistaqt
# %pip install ipywidgets
# %pip install ipyevents
# %pip install trame
# %pip install trame-vuetify
# %pip install trame-vtk
# %pip install pyvista

import mne
import numpy as np 
import matplotlib.pyplot as plt
import torch
import os
import torch.nn as nn
from mne import setup_source_space
from mne import make_forward_solution
from mne.minimum_norm import make_inverse_operator, apply_inverse
from mne.datasets import fetch_fsaverage
from mne import compute_raw_covariance

# Creating a MNE Raw object
ch_names = ['EEG {0}'.format(i) for i in range(64)] 
ch_types = ['eeg' for _ in range(64)]
info = mne.create_info(ch_names=ch_names, sfreq=160.0, ch_types=ch_types)
raw = mne.io.RawArray(np.load("./data/training/training-closed-64ch-normalized.npy")[2], info)
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
new_names = ten_twenty_montage.ch_names[:len(raw.ch_names)]
mapping = dict(zip(raw.ch_names, new_names)) # Create a dictionary mapping old names to new names then rename
raw.rename_channels(mapping)
raw.set_montage(ten_twenty_montage)

raw.set_eeg_reference('average', projection=True)

# Use pre-processing methods in MNE, for instance: filter the signal
# raw.filter(l_freq=1, h_freq=None)

# Setting the subjects_dir path
subjects_dir = str(mne.datasets.sample.data_path()) + '/subjects'

# Setting up the source space
src = setup_source_space('fsaverage', spacing='oct6', add_dist=False, subjects_dir=subjects_dir)

# Making the forward model
fsaverage = mne.datasets.fetch_fsaverage(verbose=True)
bem_dir = os.path.join(fsaverage, 'bem')
bem_fname = os.path.join(bem_dir, 'fsaverage-5120-5120-5120-bem-sol.fif')
bem_sol = mne.read_bem_solution(bem_fname, verbose=True)

# Now pass the bem_sol to the make_forward_solution function
fwd = make_forward_solution(raw.info, trans=None, src=src, bem=bem_sol, meg=False, eeg=True)

# Compute the inverse solution
noise_cov = compute_raw_covariance(raw, tmin=0.0, tmax=9760 / raw.info['sfreq']) # Compute the covariance on a segment of the raw data
inv = make_inverse_operator(raw.info, fwd, noise_cov, loose=0.2, depth=0.8)
lambda2 = 1.0 / 9.0  # this is equivalent to using a signal-to-noise ratio of 3
method = 'dSPM'  # use dSPM method (could also be MNE or sLORETA)
events = mne.make_fixed_length_events(raw, duration=2.0)  # Here duration is in sec.
epochs = mne.Epochs(raw, events, tmin=0, tmax=60, baseline=None, reject=None)  # Here tmin and tmax are in sec.
evoked = epochs.average()

stc = apply_inverse(evoked, inv, lambda2, method)
stc.plot(
    subject='fsaverage',
    hemi='both',  # Specify 'both' to show both hemispheres
    surface='inflated',
    subjects_dir=subjects_dir,
    time_viewer=True
)



ValueError: len(data) (1) does not match len(info["ch_names"]) (64)

In [13]:
open = np.load("./data/training/eyes-closed-64ch.npy")[2]


print(np.shape(open))

(64, 9760)
